In [ ]:
# deep learning with PyTorch: Neural Style Transfer
# set Google Colab runtime

# !pip install torch torchvision
# !git clone https://github.com/parth1620/Project-NST.git

# loading VSG-19 pretrained model using model subpackage in PyTorch
import torch
from torchvision import models

vgg = models.vgg19(pretrained = True)

# prints features and classifiers and their layers
# for this project, we don't use the classifiers, as we extract the content and style from the layers in the features
print(vgg)

vgg = vgg.features
# see only the features component
print(vgg)

# parse the gradient, use the pretrained weight without updating it
# freeze the model so no gradient computation occurs in the training loop
for parameters in vgg.parameters():
  parameters.requires_grad_(False)

# check and move variables and model to GPU if available, otherwise to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print to check what device was set to
print(device)
vgg.to(device)

# preprocess image
from PIL import Image
from torchvision import transforms as T

# create preprocess function
# used to resize the image if it is larger than max_size
def preprocess(img_path, max_size = 500):
  image = Image.open(img_path).convert('RGB')

  if max(image.size) > max_size:
    size = max_size
  else:
    size = max(image.size)

  # all pre-trained models expect input images normalized in the same way
  # loaded in to a range of [0, 1] and then normalized using mean and std values
  # mean and std values copied from the torchvision.models manual documentation
  img_transforms = T.Compose([
                  T.Resize(size),
                  T.ToTensor(),
                  T.Normalize(mean = [0.485, 0.456, 0.406],
                              std = [0.229, 0.224, 0.225])
  ])
  image = img_transforms(image)

  # if image in shape of ex (3, 224, 224) (c, h w) -> (1, 3, 224, 224)
  image = image.unsqueeze(0)

  return image

# pass content and style image to preprocess function
content_p = preprocess('/content/Project-NST/content11.jpg')
style_p = preprocess('/content/Project-NST/style12.jpg')

# pass to GPU
content_p = content_p.to(device)
style_p = style_p.to(device)

# print the shape
print("Content Shape:", content_p.shape)
print("Style Shape:", style_p.shape)